While in example #1 we compared two models based on our data, in this step we will choose the better model and save it, such that we can quickly push predictions to SQL Server (on a schedule, if we want).

Note: while we created and checked models in step #1, we automatically held out some of the data from your query in a test set (to see how accurate our models were). Know that we have a sense for model accuracy, in this step, we re-create the best model on your ENTIRE data set, and then use it to create predictions.

If you have SQL Server on your machine and HCRTools installed, there's just one more prerequisite: an entity to receive the predictions. Call it what you may, but it should have these columns for classification:

In [1]:
#CREATE TABLE [SAM].[dbo].[HCRDeployBASE](       --If possible, create using SAMD
#[BindingID] [int] NOT NULL,
#[BindingNM] [varchar](255) NOT NULL,
#[LastLoadDTS] [datetime2](7) NOT NULL,
#[PatientEncounterID] [decimal](38, 0) NOT NULL, --Make this col name same as in input table
#[PredictedProbNBR] [decimal](38, 2) NOT NULL,   --Change to PredictedValueNBR for regression
#[Factor1TXT] [varchar](255) NULL,
#[Factor2TXT] [varchar](255) NULL,
#[Factor3TXT] [varchar](255) NULL
#)

Now on to the code!  First, load the package and define SQL server connection string for reading in data and writing predictions 

In [2]:
library(HCRTools)

Type ?HCRTools to read the docs.


In [3]:
connection.string = 'driver={SQL Server};
                     server=localhost;
                     database=SAM;
                     trusted_connection=true'

Now read in data, either from SQL server or .csv. By default we will use the prepackaged .csv, but feel free to uncomment out the lines below to use SQL instead:

In [4]:
# Be sure to grab both test and training rows.
# query = "SELECT
#        [PatientEncounterID]
#       ,[PatientID]
#       ,[SystolicBPNBR]
#       ,[LDLNBR]
#       ,[A1CNBR]
#       ,[GenderFLG]
#       ,[ThirtyDayReadmitFLG]
#       ,[InTestWindow]
#   FROM [SAM].[dbo].[DiabetesClinicalOutpatient]

# df <- selectData(connection.string, query)

In [5]:
#From CSV

# This line will identify our prepackaged sample data for loading.  You can delete this if using your own data.
csvfile <- system.file("extdata", "DiabetesClinical.csv", package = "HCRTools")

df <- read.csv(file = csvfile, #<-- or path/to/yourfile.csv
                    header = TRUE,
                    na.strings = c('NULL', 'NA', ""))

Check the data types of the dataframe to make sure factor cols aren’t listed as numeric cols, etc.

In [6]:
str(df)

'data.frame':	1000 obs. of  8 variables:
 $ PatientEncounterID : int  1 2 3 4 5 6 7 8 9 10 ...
 $ PatientID          : int  10001 10001 10001 10002 10002 10002 10002 10003 10003 10003 ...
 $ SystolicBPNBR      : int  167 153 170 187 188 185 189 149 155 160 ...
 $ LDLNBR             : int  195 214 191 135 125 178 101 160 144 130 ...
 $ A1CNBR             : num  4.2 5 4 4.4 4.3 5 4 5 6.6 8 ...
 $ GenderFLG          : Factor w/ 2 levels "F","M": 2 2 2 2 2 2 2 2 2 2 ...
 $ ThirtyDayReadmitFLG: Factor w/ 2 levels "N","Y": 1 1 1 1 1 1 1 1 1 2 ...
 $ InTestWindowFLG    : Factor w/ 2 levels "N","Y": 1 1 1 1 1 1 1 1 1 1 ...


Change a column type, if necessary

In [7]:
df$GenderFLG  = as.factor(df$GenderFLG)
df$LDLNBR     = as.numeric(df$LDLNBR) # only here for demonstration
str(df)

'data.frame':	1000 obs. of  8 variables:
 $ PatientEncounterID : int  1 2 3 4 5 6 7 8 9 10 ...
 $ PatientID          : int  10001 10001 10001 10002 10002 10002 10002 10003 10003 10003 ...
 $ SystolicBPNBR      : int  167 153 170 187 188 185 189 149 155 160 ...
 $ LDLNBR             : num  195 214 191 135 125 178 101 160 144 130 ...
 $ A1CNBR             : num  4.2 5 4 4.4 4.3 5 4 5 6.6 8 ...
 $ GenderFLG          : Factor w/ 2 levels "F","M": 2 2 2 2 2 2 2 2 2 2 ...
 $ ThirtyDayReadmitFLG: Factor w/ 2 levels "N","Y": 1 1 1 1 1 1 1 1 1 2 ...
 $ InTestWindowFLG    : Factor w/ 2 levels "N","Y": 1 1 1 1 1 1 1 1 1 1 ...


Remove a column, if necessary. Note that typical machine learning algorithms (like RF) can't use the PatientID column, so we remove it here.

In [8]:
df$PatientID <- NULL

Note that in Example #1 we found the random forest model to work best, so let's use it to create predictions. This step will do that and push the results to SQL Server (note you choose the destination entity here).

Define Model Parameters

In [9]:
set.seed(42) # <-- used to make results reproducible
p <- SupervisedModelDeploymentParams$new()
p$type = 'classification'
p$df = df
p$grainCol = 'PatientEncounterID'
p$testWindowCol = 'InTestWindowFLG'
p$predictedCol = 'ThirtyDayReadmitFLG'
p$impute = TRUE
p$debug = FALSE
p$useSavedModel = FALSE
p$cores = 1
p$sqlConn = connection.string
p$destSchemaTable = 'dbo.HCRDeployClassificationBASE'

In [10]:
rF = RandomForestDeployment$new(p)
rF$deploy()

[1] "Details for proability model:"
Ranger result

Call:
 ranger(as.formula(paste(self$params$predictedCol, ".", sep = " ~ ")),      data = private$dfTrain, probability = TRUE, num.trees = self$params$trees,      write.forest = TRUE, mtry = rfMtryTemp) 

Type:                             Probability estimation 
Number of trees:                  201 
Sample size:                      977 
Number of independent variables:  4 
Mtry:                             2 
Target node size:                 10 
Variable importance mode:         none 
OOB prediction error:             0.08054102 
[1] "Probability predictions are based on random forest"
[1] "SQL Server insert was successful"


That's it! Your choice of model has been trained on your entire dataset, your predictions have been made, and your results now sit in SQL Server. Remember that to see the docs you can always type this in the console: 

In [11]:
library(HCRTools)
?HCRTools

Feel reach out to Levi Thatcher (levi.thatcher@healthcatalyst.com) with any questions or suggestions!